In [1]:
#custom libraries
from data_import import *
from data_preprocess import *
from data_preprocess_BERT import *

In [2]:
#set filepath constants
NLP_TOPIC_RECOGNITION_FILEPATH = '../NLP Topic Recognition'
#NLP_MODEL_FILEPATH = '../NLP Model Training/model_100K_glove'
NLP_MODEL_FILEPATH = '../NLP Model Training/model_100K_glove_91'
TEXTUAL_RAW_DATA_FILEPATH = '../Data/textual_raw'

## Glove 0.91
OUTPUT_DATASET_FILEPATH = "glove_100k_91_min.csv"
# OUTPUT_DATASET_FILEPATH = "BTC_open_close.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_91_glove_filtered_full_end_start.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_91_filtered_full_user_averaged.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_91_filtered_full_user_weights_user_averaged.csv"

## Glove standar
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_filtered_full_user_weights_user_averaged.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_filtered_full_user_weights.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_filtered_2g_user_weights.csv"

## BERT
# OUTPUT_DATASET_FILEPATH_PREV = "Dataset_sentiment_BERT_trainable.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_BERT_trainable_full.csv"

In [3]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append(NLP_TOPIC_RECOGNITION_FILEPATH)
from topic_recognition import *
#from NLP_class import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [6]:
from tensorflow import keras
model = keras.models.load_model(NLP_MODEL_FILEPATH)

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           10709300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_201 (Dense)            (None, 1)                 101       
Total params: 10,789,801
Trainable params: 80,501
Non-trainable params: 10,709,300
_________________________________________________________________


In [ ]:
# returns a dictionary of the form {'username': (sentiment_score_sum, occurances)}
def get_users(paths):
    df = pd.DataFrame()
    for path in paths:
        text_data = preprocess_textual(path)
        df = pd.concat([df, text_data])
    
    #take the set of all usernames
    ini_set = set(df['username'].values)
    #initialized as neutral (0.5) and 1 occurance
    users = dict.fromkeys(ini_set, (0.5,1))
    return users

#updates username's sentiment score values after recieving username's tweet
def user_update(username, sentiment):
    #update 
    (s,t) = users[username]
    (s_new, t_new) = (s + sentiment, t + 1)
    users[username] = (s_new, t_new)
    
    #sentiment calculation sentiment_weighted has range(-1, 1)
    #the extreme values are seen when a positive (on average) user tweets something really negative and the converse
    sentiment_weighted = sentiment - s/t
    return sentiment_weighted

In [ ]:
#Create a bert_preprocess object that automatically sets tokenizers and pretrained bert model classifier
bert = BERT_preprocess()

In [ ]:
# lol = text.df[:100]
# lol['sentiment']=bert.predict(list(lol['text'].values))
# pd.options.display.max_colwidth = 400
# print(lol[['sentiment','text']])
# lol = text.df
# list(text.df['text'].values)
# start = timer()
# lol.text.map(lambda x: bert.predict([x]))
# print(timer() - start)

# Sentiment analysis on Tweets

## BERT

In [ ]:
#fetch btc price data
btc_data = preprocess_BTC()

#itterate folder for all raw textual data
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]

#initialize username dictionary for user wieghted sentiment scores
#users = get_users(paths)

In [ ]:
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]
print(paths)

In [ ]:
i = 1

# final dataframe (initialize or load)
# df = pd.DataFrame()
df = pd.read_csv(OUTPUT_DATASET_FILEPATH, index_col = 0)
print(df)
start = timer()
for path in paths:
#     if i<47:
#         i+=1
#         continue
    message = "Iterration " + str(i) + " of " + str(len(paths))
    print(message)
    text_data = preprocess_textual(path)
    
    #initialize an NLP_preprocess object for the current dataset
    text = NLP_preprocess(text_data)
    
    #print(text.df.shape[0])
    #perform some spam_filtering on the current dataset
    text.spam_filtering()
    #print(text.df.shape[0])
    
    #calculate the sentiment score for every tweet in the current dataset
    text.df['sentiment'] = bert.predict(list(text.df['text'].values))
    
    #add an "hour" column that contains the datetime in hourly resolution
    text.df["hour"] =  text.df.date.map(lambda x: x[:-6])
    #print(text.df)
   
    #get the mean and median sentiment per hour
    timetable = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
    timetable2 = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    
    #sync the price and sentiment dataset so that on each row price follows the sentiment
    timetable = timetable[:-1]
    timetable2 = timetable2[:-1]
    btc_connected = connect_datasets(text.df, btc_data)[1:]
    
    i += 1
    #if mismatching sizes disregard current dataset
    if len(timetable.index) != len(btc_connected.index):
        continue
    #otherwise combine them into a single dataset 
    timetable.index = btc_connected.index
    timetable2.index = btc_connected.index
    timetable = pd.concat([timetable['Bitcoin'], timetable2['Bitcoin_median'], btc_connected['Open']], axis = 1)
    #print(timetable)
    
    df = pd.concat([df, timetable], axis = 0)   
    print(df)
    
    #save final dataset
    df.to_csv(OUTPUT_DATASET_FILEPATH)
print(timer() - start)


In [ ]:
df.to_csv(OUTPUT_DATASET_FILEPATH)

## Plain Glove

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [7]:
#fetch btc price data
btc_data = preprocess_BTC()

#itterate folder for all raw textual data
# txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
# paths = [x for x in txt_folder]

#initialize username dictionary for user wieghted sentiment scores
#users = get_users(paths)

In [29]:
#btc_data.to_csv('./btc_price_processed.csv')
btc_data = pd.read_csv('btc_price_processed.csv', index_col = 0)
btc_data

C:\Users\Konst\anaconda3\envs\tensorflow-gpu\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [30]:
btc_data['datetime'] = btc_data.Timestamp.apply(lambda x : datetime.datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

In [31]:
btc_data

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,datetime
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000,2011-12-31 07:52:00
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:53:00
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:54:00
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:55:00
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:56:00
...,...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339,2021-03-30 23:56:00
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508,2021-03-30 23:57:00
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496,2021-03-30 23:58:00
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202,2021-03-30 23:59:00


In [20]:
l = pd.DataFrame([[1,'2021-03-30 23:59:00'],[2,'2021-03-30 23:58:00']],columns=['x','datetime'])
l

,x,datetime
0,1,2021-03-30 23:59:00
1,2,2021-03-30 23:58:00


In [21]:
pd.merge(btc_data,l,on='datetime',how="left")

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,datetime,x
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000,2011-12-31 07:52:00,NaN
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:53:00,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:54:00,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:55:00,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:56:00,NaN
...,...,...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339,2021-03-30 23:56:00,NaN
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508,2021-03-30 23:57:00,NaN
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496,2021-03-30 23:58:00,2.0
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202,2021-03-30 23:59:00,1.0


In [35]:
#Test minute accuracy
#itterate folder for all raw textual data
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]

i = 1

# final dataframe
df = pd.DataFrame()
weighted = False

start = timer()
for path in paths:

    message = "Iterration " + str(i) + " of " + str(len(paths))
    print(message)
    text_data = preprocess_textual(path)

#     old approach
#     timetable = topic_recognition(text_data, flagged_usernames = [], display = message)
#     print(timetable)
    
    #initialize an NLP_preprocess object for the current dataset
    text = NLP_preprocess(text_data)
    
    #print(text.df.shape[0])
    #text.flag_users()
    
    
    #basic nlp preprocessing (rm stopwords, rm punctuation etc.)
    text.preprocess_data()
    #perform some spam filtering
    text.spam_filtering()
    
    #print(text.df.shape[0])
    
    #import default tokenizer for the model
    text.import_tokenizer()
    #tokenize and pad text sequences of the dataset
    df_padded = text.tokenize_and_pad(train = False)
        
    #apply the model on the padded sequences to get sentiment
    text.df["sentiment"] = (model.predict(df_padded))
    #print(text.df.index)

#     for r in np.random.randint(0,text.df.shape[0],10):
#         print(text.df.iloc[r]['text'], text.df.iloc[r]['sentiment'])


    #try: weight messeges by the average optimism/pessimism of each user
    #this might have an impact because there are about 5.5 messages per user,
    #so there is some history about them
    if weighted:
        text.df['weighted'] = text.df.index.map(lambda x: user_update(text.df.loc[x]['username'],  text.df.loc[x]['sentiment'])) 
    #print(text.df)
    #add an "hour" column that contains the datetime in hourly resolution
    text.df["minute"] =  text.df.date.map(lambda x: x[:-3]+":00")
    #print(text.df)
    #old
#     timetable = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
#     timetable2 = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    

    #and then get the mean and median weighted sentiment per hour
    if weighted:
        #we can eliminate the big effect of users that sent multiple messages by first averaging per user per hour
        user_averaged = text.df.groupby(['minute', 'username'], as_index=False).weighted.apply(np.mean)
        timetable = pd.DataFrame(user_averaged.groupby(['minute']).weighted.apply(np.mean).values, columns = ["Bitcoin"])
        timetable2 = pd.DataFrame(user_averaged.groupby(['minute']).weighted.apply(np.median).values, columns = ["Bitcoin_median"])
    else:
        #we can eliminate the big effect of users that sent multiple messages by first averaging per user per hour
        user_averaged = text.df.groupby(['minute', 'username'], as_index=False).sentiment.apply(np.mean)
#         timetable = pd.DataFrame(user_averaged.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
#         timetable2 = pd.DataFrame(user_averaged.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
        
        timetable = pd.DataFrame(text.df.groupby(['minute'], as_index=False).sentiment.apply(np.mean).values, columns = ["datetime","Bitcoin"])
        timetable2 = pd.DataFrame(text.df.groupby(['minute'], as_index=False).sentiment.apply(np.median).values, columns = ["datetime","Bitcoin_median"])
    
    #print(timetable)
    #get the mean and median weighted sentiment per hour
#     timetable = pd.DataFrame(text.df.groupby(['hour']).weighted.apply(np.mean).values, columns = ["Bitcoin"])
#     timetable2 = pd.DataFrame(text.df.groupby(['hour']).weighted.apply(np.median).values, columns = ["Bitcoin_median"])
    df = pd.concat([df, timetable], axis = 0) 
    #sync the price and sentiment dataset so that on each row price follows the sentiment
    print(df)
#     timetable = timetable[:-1]
#     timetable2 = timetable2[:-1]
    
#     #btc_connected = connect_datasets(text.df, btc_data)[1:]
    i += 1
#     #if mismatching sizes disregard current dataset
#     if len(timetable.index) != len(btc_connected.index):
#         continue
#     #otherwise combine them into a single dataset 
#     timetable.index = btc_connected.index
#     timetable2.index = btc_connected.index
#     timetable = pd.concat([timetable['Bitcoin'], timetable2['Bitcoin_median'], btc_connected['open'],btc_connected['close']], axis = 1)
#     print(timetable)
    
#     df = pd.concat([df, timetable], axis = 0)   
#     #print(df)

#merge with price
df = pd.merge(btc_data,df,on='datetime',how="left")
#save final dataset
df.to_csv(OUTPUT_DATASET_FILEPATH)
print(timer() - start)


Iterration 1 of 65
Texts containing suspicious phrases
play brand new multicoin game free bitcoin faucet bitcoingame bitcoin faucet
today free bitcoin guidebook everything need know nonfiction pictwittercomau5fpdwmpu
get free bitcoin wallet
freebitcoin free bitcoin wallet faucet lottery dice
free bitcoin cloud mining
added video youtube playlist best free bitcoin miner 0002 bitcoin day
get free bitcoin every 30 minutes 20000satoshi
15jqru8gash2gy3ibb 00000008 btc free lottery looking free btc … yabtcl bitcoin
get free bitcoin pictwittercomxoozucjpbb
cryptocurrency market trading free bitcoin 50 make money onlinemp4 world wide pump community start welcome to… pictwittercoma2lxrtxyqb
                 datetime   Bitcoin
0     2017-09-13 00:00:00  0.553036
1     2017-09-13 00:01:00  0.484907
2     2017-09-13 00:02:00  0.706784
3     2017-09-13 00:03:00  0.627043
4     2017-09-13 00:04:00  0.569503
...                   ...       ...
1435  2017-09-13 23:55:00  0.620598
1436  2017-09-13 23:5

                 datetime   Bitcoin
0     2017-09-13 00:00:00  0.553036
1     2017-09-13 00:01:00  0.484907
2     2017-09-13 00:02:00  0.706784
3     2017-09-13 00:03:00  0.627043
4     2017-09-13 00:04:00  0.569503
...                   ...       ...
1435  2017-10-16 23:55:00  0.788858
1436  2017-10-16 23:56:00   0.65858
1437  2017-10-16 23:57:00  0.745758
1438  2017-10-16 23:58:00  0.756469
1439  2017-10-16 23:59:00  0.711828

[10268 rows x 2 columns]
Iterration 9 of 65
Texts containing suspicious phrases
free bitcoins faucet earn 100 free btc 5 minutes bitcoin freebtc getbtc
free bitcoins faucet earn 100 free btc 5 minutes bitcoin freebtc getbtc
free bitcoins faucet earn 100 free btc 5 minutes bitcoin freebtc getbtc
j free bitcoin cloud mining
collecting free bitcoin gold btg coins coinomi wallet medium bitcoingold coinomi …pictwittercomiw6ujko6po
2 years ago people wouldnt even collect free bitcoin must another new ico floydmayweather
get free bitcoin
get free bitcoin pictwittercom

                 datetime   Bitcoin
0     2017-09-13 00:00:00  0.553036
1     2017-09-13 00:01:00  0.484907
2     2017-09-13 00:02:00  0.706784
3     2017-09-13 00:03:00  0.627043
4     2017-09-13 00:04:00  0.569503
...                   ...       ...
1435  2017-12-03 23:55:00  0.728243
1436  2017-12-03 23:56:00  0.594891
1437  2017-12-03 23:57:00  0.588999
1438  2017-12-03 23:58:00   0.65543
1439  2017-12-03 23:59:00  0.658086

[20348 rows x 2 columns]
Iterration 16 of 65
Texts containing suspicious phrases
14122017 x2 blockchain get free bitcoin twitter bitcoin mining software freebsd
buy sell 100 digitalcurrency via coinbase get 10 free btc bitcoin btc
earn free bitcoin faucethubio bitcoin faucet game faucets micropayments
moneycoach get free bitcoin money pictwittercomasnyhwe3nn
earn free bitcoin every 5 minute take unlimited via youtube
earn free bitcoin faucethubio bitcoin faucet game faucets micropayments
1dudpca6qxroxmnpat 00000006 btc free lottery fun way win free btc … yabtcl

                 datetime   Bitcoin
0     2017-09-13 00:00:00  0.553036
1     2017-09-13 00:01:00  0.484907
2     2017-09-13 00:02:00  0.706784
3     2017-09-13 00:03:00  0.627043
4     2017-09-13 00:04:00  0.569503
...                   ...       ...
1435  2018-03-15 23:55:00  0.515262
1436  2018-03-15 23:56:00   0.38052
1437  2018-03-15 23:57:00  0.822207
1438  2018-03-15 23:58:00  0.665433
1439  2018-03-15 23:59:00  0.623218

[28509 rows x 2 columns]
Iterration 23 of 65
Texts containing suspicious phrases
free 5 legitcoin token 24 february march priced least 80sign free bitcoin litecoin ethereum ripple monero btc ltc eth xrp xmr crypto money free legitcoin stocks freetoken blockchain altcoin ico cryptofollow
second life creator high fidelity’s hfc social cryptocurrency vr generate bitcoin take free bitcoin
get free bitcoin
cryptopia get free bitcoin money pictwittercom3ipkckredo
free bitcoin cloud mining
howdoyoutubersmakemoney get free bitcoin money pictwittercome3zilzloll
g20 conf

Texts containing suspicious phrases
get free bitcoin
earn free bitcoin get 30000 satoshi daily best cloud mining site …
get free bitcoin
free bitcoin cloud mining
1203 www startminer free bitcoin mining without beeing even online try completly free
0934 venice follow us free bitcoin information
referral links • coinlaunch airdrop 250 tokens telegram sign generate bitcoin take free bitcoin
get free bitcoin
earn free bitcoin bitcoin faucet game faucets micropayments
2338 korting start free bitcoin mining project 1000 ghs free pictwittercomdflrfqvbjh
                 datetime   Bitcoin
0     2017-09-13 00:00:00  0.553036
1     2017-09-13 00:01:00  0.484907
2     2017-09-13 00:02:00  0.706784
3     2017-09-13 00:03:00  0.627043
4     2017-09-13 00:04:00  0.569503
...                   ...       ...
1435  2018-05-11 23:55:00  0.662525
1436  2018-05-11 23:56:00  0.684961
1437  2018-05-11 23:57:00  0.686846
1438  2018-05-11 23:58:00  0.711236
1439  2018-05-11 23:59:00   0.59188

[38163 rows x

Texts containing suspicious phrases
want get started invesing crypto sign coinbase get 10 free bitcoin …
get free bitcoin
russian treasurers association joins masterchain banking pilot generate bitcoin take free bitcoin
get free bitcoin
free bitcoin cloud mining
crypto dispensers generate bitcoin take free bitcoin
sec made announcement bitcoin security generate bitcoin take free bitcoin
free bitcoins faucet earn 100 free btc 5 minutes bitcoin freebtc getbtc
get free bitcoin
new post free bitcoin generator 2017 survey cracked published top bitcoin videos …pictwittercomo3xuweslqy
                 datetime   Bitcoin
0     2017-09-13 00:00:00  0.553036
1     2017-09-13 00:01:00  0.484907
2     2017-09-13 00:02:00  0.706784
3     2017-09-13 00:03:00  0.627043
4     2017-09-13 00:04:00  0.569503
...                   ...       ...
1435  2018-06-15 23:55:00  0.586743
1436  2018-06-15 23:56:00  0.653985
1437  2018-06-15 23:57:00  0.663117
1438  2018-06-15 23:58:00  0.711262
1439  2018-06-15 23

                 datetime   Bitcoin
0     2017-09-13 00:00:00  0.553036
1     2017-09-13 00:01:00  0.484907
2     2017-09-13 00:02:00  0.706784
3     2017-09-13 00:03:00  0.627043
4     2017-09-13 00:04:00  0.569503
...                   ...       ...
1435  2018-07-17 23:55:00  0.696591
1436  2018-07-17 23:56:00  0.733994
1437  2018-07-17 23:57:00  0.781405
1438  2018-07-17 23:58:00  0.778876
1439  2018-07-17 23:59:00  0.803301

[56883 rows x 2 columns]
Iterration 43 of 65
Texts containing suspicious phrases
get free bitcoin
get free bitcoin
get free bitcoin
buy sell 100 digital currency get 10 free bitcoin bitcoin btc cryptocurrency cryptocurrencies blockchain edtech vr virtualreality 3d
fed chair cryptocurrencies great money laundering earn free bitcoin less 10 minutes bitcoin btcpeek freebitcoin btc freebtc money cryptocurrency crypto blockchain ethereum bitcoinprice getbitcoin
sec halts kodak ‘kashminer’ bitcoin mining operation earn free bitcoin less 10 minutes bitcoin btcpeek fre

                 datetime   Bitcoin
0     2017-09-13 00:00:00  0.553036
1     2017-09-13 00:01:00  0.484907
2     2017-09-13 00:02:00  0.706784
3     2017-09-13 00:03:00  0.627043
4     2017-09-13 00:04:00  0.569503
...                   ...       ...
1435  2018-08-14 23:55:00  0.692633
1436  2018-08-14 23:56:00    0.6624
1437  2018-08-14 23:57:00  0.698832
1438  2018-08-14 23:58:00  0.744772
1439  2018-08-14 23:59:00  0.540501

[65523 rows x 2 columns]
Iterration 49 of 65
Texts containing suspicious phrases
get free bitcoin
get free bitcoin …
bitcoin cloud mining free bitcoin mining investment project dealing mining trading payment proof … mining bot … bitcoin mining binance btc btcusd bitfinex bittrex btccmt telegram
hi ive using coinbase makes really easy safe buy sell store digital currency like bitcoin got us 10 free bitcoin sign invite link buy sell least 100 digital currency …
want join free bitcoin mastermind group get free bitcoin business see
get free bitcoin
get free bitcoin

Texts containing suspicious phrases
get free bitcoin
get free bitcoin
free bitcoins earn 65 free btc 5 minutes bitcoin freebtc getbtc
pivot trusted legitimate app earn free bitcoin daily …
free bitcoins earn 65 free btc 5 minutes bitcoin freebtc getbtc
ronnyonline get free bitcoin money pictwittercome2lfcx71w7
earn free bitcoin mining site
coinpot simple tricks earn free bitcoin altcoins passively everyday automate
get free bitcoin hourly play simple games solve mining captchas per 24 hrrefer friends
best fastest way get free bitcoin october 2018 fast free
                 datetime   Bitcoin
0     2017-09-13 00:00:00  0.553036
1     2017-09-13 00:01:00  0.484907
2     2017-09-13 00:02:00  0.706784
3     2017-09-13 00:03:00  0.627043
4     2017-09-13 00:04:00  0.569503
...                   ...       ...
1435  2018-10-18 23:55:00  0.714057
1436  2018-10-18 23:56:00  0.724063
1437  2018-10-18 23:57:00  0.785779
1438  2018-10-18 23:58:00  0.581535
1439  2018-10-18 23:59:00  0.445812

[756

Texts containing suspicious phrases
get free bitcoin
members already received 3k btcs since launch enter btc wallet code start mining free bitcoin
get free bitcoin
check pei start earning automatic cashback bitcoin cash existing cards use code join x61up0 download
free bitcoin cryptocurrency watch short videos get paid cryptocurrency get crypto also digibyte litecoin ethereum ripple steem bat get free cryptocurrency earncrypto zil ams nlg pictwittercomsviqhlmp9h
get 25 blockchain courses use code blockgeeks25 blockchain ethereum bitcoin pictwittercomn1stjwbrtn
cool earn bitcoin free bitcoin
get free bitcoin
play brand new multicoin game free bitcoin faucet bitcoingame bitcoin faucet
hi ive using coinbase makes really easy safe buy sell store digital currency like bitcoin got us 10 free bitcoin sign invite link buy sell least 100 digital currency …
                 datetime   Bitcoin
0     2017-09-13 00:00:00  0.553036
1     2017-09-13 00:01:00  0.484907
2     2017-09-13 00:02:00  0.706

In [71]:
first = df.dropna(axis=0).index[0]
last = df.dropna(axis=0).index[-1]
df = df[df.index >= first]
df = df[df.index <= last]
df

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,datetime,Bitcoin,hour
2992576,1505260800,4170.60,4173.90,4170.60,4173.90,2.531700,10564.728748,4172.978136,2017-09-13 00:00:00,0.553036,2017-09-13 00:00:00
2992577,1505260860,4173.64,4179.14,4173.64,4178.99,2.174010,9084.412893,4178.644390,2017-09-13 00:01:00,0.484907,2017-09-13 00:00:00
2992578,1505260920,4178.12,4178.12,4177.90,4177.90,1.347975,5631.744164,4177.929021,2017-09-13 00:02:00,0.706784,2017-09-13 00:00:00
2992579,1505260980,4171.90,4176.80,4163.04,4163.04,4.635206,19318.842257,4167.849418,2017-09-13 00:03:00,0.627043,2017-09-13 00:00:00
2992580,1505261040,4162.83,4162.83,4162.83,4162.83,0.026494,110.289893,4162.830000,2017-09-13 00:04:00,0.569503,2017-09-13 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
3708183,1548197220,3582.00,3582.00,3581.02,3581.22,0.850242,3044.972345,3581.302263,2019-01-22 22:47:00,0.517467,2019-01-22 22:00:00
3708184,1548197280,3583.30,3583.30,3583.30,3583.30,0.014342,51.390614,3583.300000,2019-01-22 22:48:00,0.831291,2019-01-22 22:00:00
3708185,1548197340,3582.90,3582.90,3578.95,3578.95,0.247214,885.669135,3582.603799,2019-01-22 22:49:00,0.551472,2019-01-22 22:00:00
3708186,1548197400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-22 22:50:00,0.752571,2019-01-22 22:00:00


In [103]:
#backfill/interpolate missing values
df[['Open','Close','High','Low','Volume_(Currency)']] = df[['Open','Close','High','Low','Volume_(Currency)']].bfill()
#df[['Open','Close','High','Low','Volume_(Currency)']] = df[['Open','Close','High','Low','Volume_(Currency)']].interpolate()
df

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,datetime,Bitcoin,hour
2992576,1505260800,4170.60,4173.900,4170.60,4173.90,2.531700,10564.728748,4172.978136,2017-09-13 00:00:00,0.553036,2017-09-13 00:00:00
2992577,1505260860,4173.64,4179.140,4173.64,4178.99,2.174010,9084.412893,4178.644390,2017-09-13 00:01:00,0.484907,2017-09-13 00:00:00
2992578,1505260920,4178.12,4178.120,4177.90,4177.90,1.347975,5631.744164,4177.929021,2017-09-13 00:02:00,0.706784,2017-09-13 00:00:00
2992579,1505260980,4171.90,4176.800,4163.04,4163.04,4.635206,19318.842257,4167.849418,2017-09-13 00:03:00,0.627043,2017-09-13 00:00:00
2992580,1505261040,4162.83,4162.830,4162.83,4162.83,0.026494,110.289893,4162.830000,2017-09-13 00:04:00,0.569503,2017-09-13 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
3708183,1548197220,3582.00,3582.000,3581.02,3581.22,0.850242,3044.972345,3581.302263,2019-01-22 22:47:00,0.517467,2019-01-22 22:00:00
3708184,1548197280,3583.30,3583.300,3583.30,3583.30,0.014342,51.390614,3583.300000,2019-01-22 22:48:00,0.831291,2019-01-22 22:00:00
3708185,1548197340,3582.90,3582.900,3578.95,3578.95,0.247214,885.669135,3582.603799,2019-01-22 22:49:00,0.551472,2019-01-22 22:00:00
3708186,1548197400,3584.43,3583.665,3581.69,3584.43,NaN,82.323568,NaN,2019-01-22 22:50:00,0.752571,2019-01-22 22:00:00


In [92]:
# from minutes to higher timeframes (hours)
df["hour"] =  df.datetime.map(lambda x: x[:-6]+":00:00")
sentiment = df.groupby(['hour']).Bitcoin.apply(np.mean)
open1 = pd.DataFrame(df.groupby(['hour']).Open.first())
close = pd.DataFrame(df.groupby(['hour']).Close.last())
volume = pd.DataFrame(df.groupby(['hour'])['Volume_(Currency)'].sum())
fin = pd.merge(open1,close,on='hour')
fin = pd.merge(fin,volume,on='hour')
fin = pd.merge(fin,sentiment,on='hour')

In [94]:
fin

,Open,Close,Volume_(Currency),Bitcoin
hour,,,,
2017-09-13 00:00:00,4170.60,4138.26,2.361910e+06,0.614810
2017-09-13 01:00:00,4142.25,4105.13,1.198324e+06,0.632149
2017-09-13 02:00:00,4101.49,4026.02,4.199565e+06,0.610243
2017-09-13 03:00:00,4035.48,3945.00,7.644898e+06,0.567887
2017-09-13 04:00:00,3951.62,4010.00,5.031894e+06,0.632717
...,...,...,...,...
2019-01-22 18:00:00,3591.48,3590.67,8.305921e+05,0.674075
2019-01-22 19:00:00,3589.30,3579.28,2.188945e+06,0.653272
2019-01-22 20:00:00,3579.28,3585.51,4.748668e+05,0.673593


In [ ]:
#itterate folder for all raw textual data
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]

i = 1

# final dataframe
df = pd.DataFrame()
weighted = False

start = timer()
for path in paths:

    message = "Iterration " + str(i) + " of " + str(len(paths))
    print(message)
    text_data = preprocess_textual(path)
    
#     old approach
#     timetable = topic_recognition(text_data, flagged_usernames = [], display = message)
#     print(timetable)
    
    #initialize an NLP_preprocess object for the current dataset
    text = NLP_preprocess(text_data)
    
    #print(text.df.shape[0])
    text.flag_users()
    
    
    #basic nlp preprocessing (rm stopwords, rm punctuation etc.)
    text.preprocess_data()
    #perform some spam filtering
    text.spam_filtering()
    
    #print(text.df.shape[0])
    
    #import default tokenizer for the model
    text.import_tokenizer()
    #tokenize and pad text sequences of the dataset
    df_padded = text.tokenize_and_pad(train = False)
        
    #apply the model on the padded sequences to get sentiment
    text.df["sentiment"] = (model.predict(df_padded))
    #print(text.df.index)
    
#     for r in np.random.randint(0,text.df.shape[0],10):
#         print(text.df.iloc[r]['text'], text.df.iloc[r]['sentiment'])


    #try: weight messeges by the average optimism/pessimism of each user
    #this might have an impact because there are about 5.5 messages per user,
    #so there is some history about them
    if weighted:
        text.df['weighted'] = text.df.index.map(lambda x: user_update(text.df.loc[x]['username'],  text.df.loc[x]['sentiment'])) 
    #print(text.df)
    #add an "hour" column that contains the datetime in hourly resolution
    text.df["hour"] =  text.df.date.map(lambda x: x[:-6])
    
    #old
#     timetable = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
#     timetable2 = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    

    #and then get the mean and median weighted sentiment per hour
    if weighted:
        #we can eliminate the big effect of users that sent multiple messages by first averaging per user per hour
        user_averaged = text.df.groupby(['hour', 'username'], as_index=False).weighted.apply(np.mean)
        timetable = pd.DataFrame(user_averaged.groupby(['hour']).weighted.apply(np.mean).values, columns = ["Bitcoin"])
        timetable2 = pd.DataFrame(user_averaged.groupby(['hour']).weighted.apply(np.median).values, columns = ["Bitcoin_median"])
    else:
        #we can eliminate the big effect of users that sent multiple messages by first averaging per user per hour
        user_averaged = text.df.groupby(['hour', 'username'], as_index=False).sentiment.apply(np.mean)
#         timetable = pd.DataFrame(user_averaged.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
#         timetable2 = pd.DataFrame(user_averaged.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
        
        timetable = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
        timetable2 = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    
    #get the mean and median weighted sentiment per hour
#     timetable = pd.DataFrame(text.df.groupby(['hour']).weighted.apply(np.mean).values, columns = ["Bitcoin"])
#     timetable2 = pd.DataFrame(text.df.groupby(['hour']).weighted.apply(np.median).values, columns = ["Bitcoin_median"])
     
    #sync the price and sentiment dataset so that on each row price follows the sentiment
    timetable = timetable[:-1]
    timetable2 = timetable2[:-1]
    btc_connected = connect_datasets(text.df, btc_data)[1:]
    
    i += 1
    #if mismatching sizes disregard current dataset
    if len(timetable.index) != len(btc_connected.index):
        continue
    #otherwise combine them into a single dataset 
    timetable.index = btc_connected.index
    timetable2.index = btc_connected.index
    timetable = pd.concat([timetable['Bitcoin'], timetable2['Bitcoin_median'], btc_connected['open'],btc_connected['close']], axis = 1)
    print(timetable)
    
    df = pd.concat([df, timetable], axis = 0)   
    #print(df)

#save final dataset
df.to_csv(OUTPUT_DATASET_FILEPATH)
print(timer() - start)


# Evaluation and Testing (not important for now)

In [ ]:
df2 = pd.read_csv(OUTPUT_DATASET_FILEPATH, index_col = 0)

In [ ]:
DATASET_FILEPATH = "./Dataset_sentiment_100K_glove_filtered_2g.csv"
df2 = pd.read_csv(OUTPUT_DATASET_FILEPATH, index_col = 0)
fig = plt.figure(figsize=(18, 10))
plt.plot(range(0,100), -1*df2['Bitcoin'].values[:100])

In [ ]:
fig = plt.figure(figsize=(18, 10))
x = -df2['Bitcoin'][100:200]
x = x/x[0]
y = df2['Open'][100:200]
y = y/y[0]
plt.plot(range(0,100), x)
plt.plot(range(0,100), y)


In [ ]:
fig = plt.figure(figsize=(18, 10))
plt.plot(range(0,100), df.pct_change()['Open'].values[100:200])

In [ ]:
import random
i = random.randint(0,len(text_data.index))
print(text_data.loc[i]['text'])
print(text_data.loc[i]['sentiment'])

In [ ]:
#BTC price data of the next hour standarized
Y = pd.DataFrame(df['Open']).astype(float)
#Y = dataset_standarization(Y)

#Textual data ('Bitcoin' topic popularity) of the last hour standarized
X = pd.DataFrame(df['Bitcoin']).astype(float)
#X = dataset_standarization(X)
Y.index = X.index

X = sm.add_constant(X)

model = sm.OLS(Y, X, missing = 'drop').fit()

In [ ]:
model.summary()